In [1]:
import pandas as pd
import numpy as np
import sklearn as skl
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import category_encoders
import math

PREDICCION_REAL = False

In [2]:
#APERTURA DE ARCHIVO DE ARCHIVOS
entrenamiento_temp = pd.read_csv("../Train_TP2_Datos_2020-2C.csv")
entrenamiento_temp = entrenamiento_temp[( entrenamiento_temp['Stage'] == 'Closed Won') | ( entrenamiento_temp['Stage'] == 'Closed Lost')]
entrenamiento_temp = entrenamiento_temp.loc[(entrenamiento_temp["ASP_Currency"] == entrenamiento_temp["Total_Taxable_Amount_Currency"])]
#entrenamiento = entrenamiento.loc[entrenamiento["Total_Taxable_Amount"] > 0]

test = pd.read_csv("../Test_TP2_Datos_2020-2C.csv")

In [3]:
#FORMATO FECHAS

#Respalda fecha, usada para separa entrenamiento y test
entrenamiento_temp['Fecha'] = pd.to_datetime(entrenamiento_temp['Opportunity_Created_Date'])
columnas_fecha = ['Month','Last_Modified_Date','Account_Created_Date','Opportunity_Created_Date','Quote_Expiry_Date','Planned_Delivery_Start_Date','Planned_Delivery_End_Date']

def formato_fechas(x):
    for columna in columnas_fecha:
        x[columna] = pd.to_datetime(x[columna])
        
formato_fechas(entrenamiento_temp)
if(PREDICCION_REAL): 
    formato_fechas(test)

In [4]:
#DIVISION ENTRE SET DE ENTRENAMIENTO Y SET DE TEST

if(PREDICCION_REAL):
    entrenamiento = entrenamiento_temp.loc[entrenamiento_temp['Fecha'].dt.year <= 2017].copy()
    validacion    = entrenamiento_temp.loc[(entrenamiento_temp['Fecha'].dt.year > 2017) & (entrenamiento_temp['Fecha'].dt.month < 6)].copy()
else:
    entrenamiento = entrenamiento_temp.loc[entrenamiento_temp['Fecha'].dt.year <= 2017].copy()
    validacion    = entrenamiento_temp.loc[(entrenamiento_temp['Fecha'].dt.year > 2017) & (entrenamiento_temp['Fecha'].dt.month < 6)].copy()
    test          = entrenamiento_temp.loc[(entrenamiento_temp['Fecha'].dt.year > 2017) & (entrenamiento_temp['Fecha'].dt.month >= 6)].copy()
    entrenamiento_label = (entrenamiento['Stage'] == 'Closed Won').astype(int)
    test_label          = (test['Stage'] == 'Closed Won').astype(int)

del entrenamiento_temp

In [5]:
#LIMPIEZA

entrenamiento = entrenamiento.drop(columns=['ASP_(converted)_Currency','Quote_Type','Brand','Product_Type','Size','Product_Category_B','Price','Currency','Last_Activity','Actual_Delivery_Date','Prod_Category_A'])
test = test.drop(columns=['ASP_(converted)_Currency','Quote_Type','Brand','Product_Type','Size','Product_Category_B','Price','Currency','Last_Activity','Actual_Delivery_Date','Prod_Category_A'])
validacion = validacion.drop(columns=['ASP_(converted)_Currency','Quote_Type','Brand','Product_Type','Size','Product_Category_B','Price','Currency','Last_Activity','Actual_Delivery_Date','Prod_Category_A'])

In [6]:
#NUEVOS FEATURES

In [7]:
#Agrego feature: Duracion de la oportunidad
entrenamiento['Opportunity_Duration'] = (entrenamiento['Last_Modified_Date'] - entrenamiento['Opportunity_Created_Date']) / np.timedelta64(1, 'D')
test['Opportunity_Duration'] = (test['Last_Modified_Date'] - test['Opportunity_Created_Date']) / np.timedelta64(1, 'D')
validacion['Opportunity_Duration'] = (validacion['Last_Modified_Date'] - validacion['Opportunity_Created_Date']) / np.timedelta64(1, 'D')
#Agrego feature: Total_Amount_USD
entrenamiento["Total_Amount_USD"] = entrenamiento["Total_Amount"] * entrenamiento["ASP_(converted)"] / entrenamiento["ASP"]
test["Total_Amount_USD"] = test["Total_Amount"] * test["ASP_(converted)"] / test["ASP"]
validacion["Total_Amount_USD"] = validacion["Total_Amount"] * validacion["ASP_(converted)"] / validacion["ASP"]
#Agrego feature: Total_Taxable_Amount_USD
entrenamiento["Total_Taxable_Amount_USD"] = entrenamiento["Total_Taxable_Amount"] * entrenamiento["ASP_(converted)"] / entrenamiento["ASP"]
test["Total_Taxable_Amount_USD"] = test["Total_Taxable_Amount"] * test["ASP_(converted)"] / test["ASP"]
validacion["Total_Taxable_Amount_USD"] = validacion["Total_Taxable_Amount"] * validacion["ASP_(converted)"] / validacion["ASP"]
#Agrego feature: Total_Amount_sobre_Total_Taxable_Amount
entrenamiento["Total_Amount_sobre_Total_Taxable_Amount"] = entrenamiento["Total_Amount_USD"] / entrenamiento["Total_Taxable_Amount_USD"]
test["Total_Amount_sobre_Total_Taxable_Amount"] = test["Total_Amount_USD"] / test["Total_Taxable_Amount_USD"]
validacion["Total_Amount_sobre_Total_Taxable_Amount"] = validacion["Total_Amount_USD"] / validacion["Total_Taxable_Amount_USD"]

In [8]:
#FEATURE - Duracion por familia
df_zona = entrenamiento[['Stage','Region','Territory','Product_Family','Planned_Delivery_Start_Date']]
df_zona = df_zona[df_zona['Stage'] == 'Closed Won']
df_familia = df_zona.groupby(['Product_Family'])['Planned_Delivery_Start_Date'].agg(['max','min']).reset_index()
df_familia['Duracion'] = (df_familia['max'] - df_familia['min']).dt.days
df_familia.columns = ['Product_Family','Planed_Delivery_Fecha_Max','min','Duracion_Familia']
entrenamiento = entrenamiento.merge(df_familia[['Product_Family','Duracion_Familia','Planed_Delivery_Fecha_Max']],on='Product_Family',how='left')
entrenamiento['Vida_Util_Ventaja'] =  (entrenamiento['Planned_Delivery_Start_Date'] - entrenamiento['Planed_Delivery_Fecha_Max']).dt.days - entrenamiento['Duracion_Familia']
entrenamiento = entrenamiento.drop('Planed_Delivery_Fecha_Max',1)
test = test.merge(entrenamiento[['Product_Family','Duracion_Familia','Vida_Util_Ventaja']].drop_duplicates(subset=['Product_Family']),left_on='Product_Family',right_on='Product_Family',how='left')
validacion = validacion.merge(entrenamiento[['Product_Family','Duracion_Familia','Vida_Util_Ventaja']].drop_duplicates(subset=['Product_Family']),left_on='Product_Family',right_on='Product_Family',how='left')

#FEATURE - Duracion por region
df_zona = entrenamiento[['Stage','Region','Territory','Product_Family','Planned_Delivery_Start_Date']]
df_zona = df_zona[df_zona['Stage'] == 'Closed Won']
df_region = df_zona.groupby(['Region'])['Planned_Delivery_Start_Date'].agg(['max','min']).reset_index()
df_region['Duracion'] = (df_region['max'] - df_region['min']).dt.days
df_region.columns = ['Region','Region_Planed_Delivery_Fecha_Max','Region_min','Duracion_Region']
entrenamiento = entrenamiento.merge(df_region[['Region','Duracion_Region','Region_Planed_Delivery_Fecha_Max']],on='Region',how='left')
entrenamiento['Region_Vida_Util_Ventaja'] =  (entrenamiento['Planned_Delivery_Start_Date'] - entrenamiento['Region_Planed_Delivery_Fecha_Max']).dt.days - entrenamiento['Duracion_Region']
entrenamiento = entrenamiento.drop('Region_Planed_Delivery_Fecha_Max',1)
test = test.merge(entrenamiento[['Region','Duracion_Region','Region_Vida_Util_Ventaja']].drop_duplicates(subset=['Region']),left_on='Region',right_on='Region',how='left')
validacion = validacion.merge(entrenamiento[['Region','Duracion_Region','Region_Vida_Util_Ventaja']].drop_duplicates(subset=['Region']),left_on='Region',right_on='Region',how='left')

#FEATURE - Duracion por territorio
df_zona = entrenamiento[['Stage','Region','Territory','Product_Family','Planned_Delivery_Start_Date']]
df_zona = df_zona[df_zona['Stage'] == 'Closed Won']
df_territorio = df_zona.groupby(['Territory'])['Planned_Delivery_Start_Date'].agg(['max','min']).reset_index()
df_territorio['Duracion'] = (df_territorio['max'] - df_territorio['min']).dt.days
df_territorio.columns = ['Territory','Territory_Planed_Delivery_Fecha_Max','Territory_min','Duracion_Territory']
entrenamiento = entrenamiento.merge(df_territorio[['Territory','Duracion_Territory','Territory_Planed_Delivery_Fecha_Max']],on='Territory',how='left')
entrenamiento['Territory_Vida_Util_Ventaja'] =  (entrenamiento['Planned_Delivery_Start_Date'] - entrenamiento['Territory_Planed_Delivery_Fecha_Max']).dt.days - entrenamiento['Duracion_Territory']
entrenamiento = entrenamiento.drop('Territory_Planed_Delivery_Fecha_Max',1)
test = test.merge(entrenamiento[['Territory','Duracion_Territory','Territory_Vida_Util_Ventaja']].drop_duplicates(subset=['Territory']),left_on='Territory',right_on='Territory',how='left')
validacion = validacion.merge(entrenamiento[['Territory','Duracion_Territory','Territory_Vida_Util_Ventaja']].drop_duplicates(subset=['Territory']),left_on='Territory',right_on='Territory',how='left')

In [9]:
#FECHAS A DIAS
def fecha_a_dias(x):
    for columna in columnas_fecha:
        x[columna] = x[columna].apply(lambda x : (x - pd.to_datetime('01/01/2000', format='%m/%d/%Y')).days)

fecha_a_dias(entrenamiento)
fecha_a_dias(test)
fecha_a_dias(validacion)

In [10]:
"""Posibles nuevos features experimentales"""
"""
def mean_encoding(train,test,col_group,col_mean,operacion):
    
    codificaciones = dict()
    nombre_col = col_group + "_" + col_mean + "_" + operacion

    last_one = train.groupby(col_group).tail(1)
    for (idx, reg) in zip(last_one[col_group].index, last_one[col_group].values):
        codificaciones[reg] = (nombre_col, idx)

    train[nombre_col] = train.groupby(col_group)[col_mean].transform(operacion)    

    #Llenamos los NaN generados por cumsum con ceros.
    train.fillna(0,inplace = True)

    #Guardamos la codificacion de cada categoria segun su nombre.
    for k, v in codificaciones.items():
        col = v[0]
        idx = v[1]
        codificaciones[k] = train.loc[idx, col]
    
    # Utilizo las ultimas codificaciones de cada categoria del train set para codificar el test set.
    # Para eso utilizo el diccionario de codificaciones.

    #columnas_categoricas = x_test.select_dtypes(include='category').columns

    test[nombre_col] = test[col_group].astype(object)
    for (idx, reg) in zip(test[nombre_col].index, test[nombre_col]):
        if (reg in codificaciones):
            test.loc[idx, nombre_col] = codificaciones[reg]
        else:
            #Codifico como cero, se puede mejorar
            test.loc[idx, nombre_col] = 0
    test[nombre_col] = test[nombre_col].astype(float)

    

columnas_cat = ["Region","Territory","Bureaucratic_Code","Billing_Country","Account_Type","Opportunity_Type","Delivery_Terms","Last_Modified_By","Product_Family","Product_Name","ASP_Currency"]

columnas_num = ["Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved","Bureaucratic_Code_0_Approval","Bureaucratic_Code_0_Approved","Submitted_for_Approval","ASP","ASP_(converted)","TRF","Total_Amount_USD","Total_Taxable_Amount_USD","Opportunity_Duration"]
i = 0
for col_cat in columnas_cat:
    for col_num in columnas_num:
        print(i)
        i+= 1
        mean_encoding(entrenamiento.copy(),validacion,col_cat,col_num,"mean")
        mean_encoding(entrenamiento.copy(),validacion,col_cat,col_num,"std")
        mean_encoding(entrenamiento,test,col_cat,col_num,"mean")
        mean_encoding(entrenamiento,test,col_cat,col_num,"std")
        

def encoding_categorico(train,test,col_group,col_mean):
    
    codificaciones = dict()

    last_one = train.groupby(col_group).tail(1)
    for (idx, reg) in zip(last_one[col_group].index, last_one[col_group].values):
        codificaciones[reg] = (col_group + "_" + col_mean, idx)

    train[col_group + "_" + col_mean] = train.groupby(col_group)[col_mean].unique()    

    #Llenamos los NaN generados por cumsum con ceros.
    train.fillna(0,inplace = True)

    #Guardamos la codificacion de cada categoria segun su nombre.
    for k, v in codificaciones.items():
        col = v[0]
        idx = v[1]
        codificaciones[k] = train.loc[idx, col]
    
    # Utilizo las ultimas codificaciones de cada categoria del train set para codificar el test set.
    # Para eso utilizo el diccionario de codificaciones.

    #columnas_categoricas = x_test.select_dtypes(include='category').columns

    test[col_group + "_" + col_mean] = test[col_group].astype(object)
    for (idx, reg) in zip(test[col_group + "_" + col_mean].index, test[col_group + "_" + col_mean]):
        if (reg in codificaciones):
            test.loc[idx, col_group + "_" + col_mean] = codificaciones[reg]
        else:
            #Codifico como cero, se puede mejorar
            test.loc[idx, col_group + "_" + col_mean] = 0
    test[col_group + "_" + col_mean] = test[col_group + "_" + col_mean].astype(float)

for col1,col2 in list(permutations(columnas_cat,2)):
    encoding_categorico(entrenamiento.copy(),validacion,col1,col2)
    encoding_categorico(entrenamiento,test,col1,col2)
    
"""

'\ndef mean_encoding(train,test,col_group,col_mean,operacion):\n    \n    codificaciones = dict()\n    nombre_col = col_group + "_" + col_mean + "_" + operacion\n\n    last_one = train.groupby(col_group).tail(1)\n    for (idx, reg) in zip(last_one[col_group].index, last_one[col_group].values):\n        codificaciones[reg] = (nombre_col, idx)\n\n    train[nombre_col] = train.groupby(col_group)[col_mean].transform(operacion)    \n\n    #Llenamos los NaN generados por cumsum con ceros.\n    train.fillna(0,inplace = True)\n\n    #Guardamos la codificacion de cada categoria segun su nombre.\n    for k, v in codificaciones.items():\n        col = v[0]\n        idx = v[1]\n        codificaciones[k] = train.loc[idx, col]\n    \n    # Utilizo las ultimas codificaciones de cada categoria del train set para codificar el test set.\n    # Para eso utilizo el diccionario de codificaciones.\n\n    #columnas_categoricas = x_test.select_dtypes(include=\'category\').columns\n\n    test[nombre_col] = tes

In [11]:
"""
#CATEGORICAS A NUMERICAS - PROMEDIO"""
"""Se debe pasar train y test ordenados """
"""
def expansion_mean_encoding(columnas_categoricas,train,test,label):
    #Dividimos el dataset de entrenamiento en features y labels
    #Armo un df extra que me ayudara para codificar las categoricas.
    #x_y_train = filtrado.iloc[:-test_rows]
    #x_train = x_y_train.drop('Stage', axis=1)
    #y_train = x_y_train['Stage'].to_frame()
    #x_test = filtrado.iloc[-test_rows:].drop('Stage', axis=1)
    #y_test = filtrado.iloc[-test_rows:]['Stage'].to_frame()

    #En el set de train.
    #columnas_categoricas = x_train.select_dtypes(include='category').columns

    codificaciones = dict()

    for col in columnas_categoricas:
        last_one = train.groupby(col).tail(1)
        for (idx, reg) in zip(last_one[col].index, last_one[col].values):
            codificaciones[reg] = (col, idx)
        cumulative_sum = train.groupby(col)[label].cumsum() - train[label]
        cumulative_count = train.groupby(col).cumcount()
        train[col] = cumulative_sum/cumulative_count

    #Llenamos los NaN generados por cumsum con ceros.
    train.fillna(0,inplace = True)

    #Guardamos la codificacion de cada categoria segun su nombre.
    for k, v in codificaciones.items():
        col = v[0]
        idx = v[1]
        codificaciones[k] = train.loc[idx, col]
    
    # Utilizo las ultimas codificaciones de cada categoria del train set para codificar el test set.
    # Para eso utilizo el diccionario de codificaciones.

    #columnas_categoricas = x_test.select_dtypes(include='category').columns

    for col in columnas_categoricas:
        test[col] = test[col].astype(object)
        for (idx, reg) in zip(test[col].index, test[col]):
            if (reg in codificaciones):
                test.loc[idx, col] = codificaciones[reg]
            else:
                #Codifico como cero, se puede mejorar
                test.loc[idx, col] = 0
        test[col] = test[col].astype(float)
        
columnas_categoricas = list(entrenamiento.select_dtypes(include=['object']).columns)
if 'Stage' in columnas_categoricas : columnas_categoricas.remove('Stage')
entrenamiento["label"] = (entrenamiento['Stage'] == 'Closed Won').astype(int)
entrenamiento.sort_values("Fecha")
expansion_mean_encoding(columnas_categoricas,entrenamiento.copy().sort_values("Fecha"),validacion,"label")
expansion_mean_encoding(columnas_categoricas,entrenamiento,test,"label")
entrenamiento = entrenamiento.drop(columns='label')
"""

'\ndef expansion_mean_encoding(columnas_categoricas,train,test,label):\n    #Dividimos el dataset de entrenamiento en features y labels\n    #Armo un df extra que me ayudara para codificar las categoricas.\n    #x_y_train = filtrado.iloc[:-test_rows]\n    #x_train = x_y_train.drop(\'Stage\', axis=1)\n    #y_train = x_y_train[\'Stage\'].to_frame()\n    #x_test = filtrado.iloc[-test_rows:].drop(\'Stage\', axis=1)\n    #y_test = filtrado.iloc[-test_rows:][\'Stage\'].to_frame()\n\n    #En el set de train.\n    #columnas_categoricas = x_train.select_dtypes(include=\'category\').columns\n\n    codificaciones = dict()\n\n    for col in columnas_categoricas:\n        last_one = train.groupby(col).tail(1)\n        for (idx, reg) in zip(last_one[col].index, last_one[col].values):\n            codificaciones[reg] = (col, idx)\n        cumulative_sum = train.groupby(col)[label].cumsum() - train[label]\n        cumulative_count = train.groupby(col).cumcount()\n        train[col] = cumulative_sum/cu

In [12]:
#CATEGORICAS A NUMERICAS  - ORDINAL
def categoricas_a_numericas(train,test,label,usar_label):
    if (usar_label):
        columnas_object = list(train.select_dtypes(include=['object']).columns)
    else:
        columnas_object = list(test.select_dtypes(include=['object']).columns)
    if 'Stage' in columnas_object : columnas_object.remove('Stage')
    ohe = category_encoders.cat_boost.CatBoostEncoder(cols = columnas_object,return_df = True)
    ohe.fit(train,label)
    if (usar_label):
        columnas = ohe.transform(train,label)
        for columna in columnas_object:
            train[columna] = columnas[columna].copy()
    else:
        columnas = ohe.transform(test)
        for columna in columnas_object:
            test[columna] = columnas[columna].copy()
categoricas_a_numericas(entrenamiento,test,entrenamiento_label,False)
categoricas_a_numericas(entrenamiento,validacion,entrenamiento_label,False)
categoricas_a_numericas(entrenamiento,test,entrenamiento_label,True)

In [13]:
def convertir_a_int(x):
    columnas_float = list(x.select_dtypes(include=[np.float]).columns)
    print(columnas_float)
    for columna in columnas_float:
        x.replace([np.inf, -np.inf], np.nan,inplace=True)
        copia = x[[columna]].copy().dropna(how="all")
        x[columna] = (copia[columna]*100).astype(int)

entrenamiento = entrenamiento.loc[entrenamiento["Opportunity_Duration"] != math.inf]
test = test.loc[test["Opportunity_Duration"] != math.inf]
convertir_a_int(entrenamiento)
convertir_a_int(test)
convertir_a_int(validacion)

['Region', 'Territory', 'Bureaucratic_Code', 'Source ', 'Billing_Country', 'Account_Name', 'Opportunity_Name', 'Sales_Contract_No', 'Account_Owner', 'Opportunity_Owner', 'Account_Type', 'Opportunity_Type', 'Delivery_Terms', 'Quote_Expiry_Date', 'Last_Modified_By', 'Product_Family', 'Product_Name', 'ASP_Currency', 'ASP', 'ASP_(converted)', 'Planned_Delivery_End_Date', 'Delivery_Quarter', 'Total_Amount_Currency', 'Total_Amount', 'Total_Taxable_Amount_Currency', 'Total_Taxable_Amount', 'Opportunity_Duration', 'Total_Amount_USD', 'Total_Taxable_Amount_USD', 'Total_Amount_sobre_Total_Taxable_Amount', 'Duracion_Familia', 'Vida_Util_Ventaja', 'Duracion_Territory', 'Territory_Vida_Util_Ventaja']
['Region', 'Territory', 'Bureaucratic_Code', 'Source ', 'Billing_Country', 'Account_Name', 'Opportunity_Name', 'Sales_Contract_No', 'Account_Owner', 'Opportunity_Owner', 'Account_Type', 'Opportunity_Type', 'Delivery_Terms', 'Quote_Expiry_Date', 'Last_Modified_By', 'Product_Family', 'Product_Name', 'ASP

In [14]:
#PRODUCCION DE ARCHIVOS INTERMEDIOS
entrenamiento.to_csv("entrenamiento-listo.csv",index=False)
test.to_csv("test-listo.csv",index=False)
validacion.to_csv("validacion-listo.csv",index=False)

In [16]:
entrenamiento["Region"].value_counts()

57.0    965
50.0    951
59.0    906
56.0    802
53.0    710
       ... 
5.0       4
15.0      4
7.0       3
77.0      2
74.0      1
Name: Region, Length: 75, dtype: int64